In [ ]:
pip install "pymongo[srv]"


In [ ]:
pip install pyngrok pydantic[email] "pymongo[srv]" motor

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, EmailStr
from motor.motor_asyncio import AsyncIOMotorClient
from pyngrok import ngrok
from threading import Thread
import nest_asyncio, uvicorn, time, subprocess

MONGO_URI = uri = "Your_Monogo_URI"

app = FastAPI()

client = AsyncIOMotorClient(MONGO_URI)
db = client[DB_NAME]
users = db[USERS_COLLECTION]

# ---------- Models ----------

class UserCreate(BaseModel):
    name: str
    email: EmailStr
    age: int

class UserUpdate(BaseModel):
    name: str | None = None
    age: int | None = None

def serialize(u):
    u["_id"] = str(u["_id"])
    return u

# ---------- Routes ----------

@app.post("/users")
async def create_user(user: UserCreate):
    if await users.find_one({"email": user.email}):
        raise HTTPException(400, "Email already exists")
    r = await users.insert_one(user.dict())
    return serialize(await users.find_one({"_id": r.inserted_id}))

@app.get("/users/{email}")
async def get_user(email: str):
    u = await users.find_one({"email": email})
    if not u:
        raise HTTPException(404, "User not found")
    return serialize(u)

@app.put("/users/{email}")
async def update_user(email: str, data: UserUpdate):
    upd = {k: v for k, v in data.dict().items() if v is not None}
    if not upd:
        raise HTTPException(400, "Nothing to update")
    if not await users.find_one({"email": email}):
        raise HTTPException(404, "User not found")
    await users.update_one({"email": email}, {"$set": upd})
    return serialize(await users.find_one({"email": email}))

@app.delete("/users/{email}")
async def delete_user(email: str):
    if not await users.find_one({"email": email}):
        raise HTTPException(404, "User not found")
    await users.delete_one({"email": email})
    return {"status": "deleted"}

# ---------- Server Runner ----------

def start():
    uvicorn.run(app, host="0.0.0.0", port=8010)

if __name__ == "__main__":
    nest_asyncio.apply()
    ngrok.set_auth_token("36PWvAasARbT6ohsC6mPGODUsJg_3SQFWde2bLVSWbrMPhAjd")
    ngrok.kill()

    public = ngrok.connect(8010)
    print("PUBLIC URL:", public.public_url)
    print("DOCS:", public.public_url + "/docs")

    Thread(target=start, daemon=True).start()
    while True:
        time.sleep(1)


INFO:     Started server process [379]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8010 (Press CTRL+C to quit)


PUBLIC URL: https://f3de73bd7df3.ngrok-free.app
DOCS: https://f3de73bd7df3.ngrok-free.app/docs


/tmp/ipython-input-1827411712.py:37: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  r = await users.insert_one(user.dict())


INFO:     34.16.170.138:0 - "POST /users HTTP/1.1" 200 OK
INFO:     34.16.170.138:0 - "POST /users HTTP/1.1" 400 Bad Request
INFO:     34.16.170.138:0 - "POST /users HTTP/1.1" 200 OK
INFO:     34.16.170.138:0 - "POST /users HTTP/1.1" 400 Bad Request
INFO:     34.16.170.138:0 - "GET /users/elan%40gmail.com HTTP/1.1" 200 OK


/tmp/ipython-input-1827411712.py:49: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  upd = {k: v for k, v in data.dict().items() if v is not None}


INFO:     34.16.170.138:0 - "PUT /users/elan%40gmail.com HTTP/1.1" 200 OK
INFO:     34.16.170.138:0 - "DELETE /users/elan%40gmail.com HTTP/1.1" 200 OK


#Post

In [ ]:
import requests

r = requests.post(f"{BASE}/users", json={
    "name": "Minga",
    "email": "elango@gmail.com",
    "age": 21
})

print(r.status_code)
print(r.json())


#Get

In [ ]:
r = requests.get(f"{BASE}/users/elan@gmail.com")
print(r.status_code)
print(r.json())


#Update

In [ ]:
r = requests.put(f"{BASE}/users/elan@gmail.com", json={
    "age": 23
})

print(r.status_code)
print(r.json())


#Delete

In [ ]:
r = requests.delete(f"{BASE}/users/elan@gmail.com")
print(r.status_code)
print(r.json())
